<a href="https://colab.research.google.com/github/s295103/Tweetsumm/blob/main/Tweetsumm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Tweetsumm and COMET repositories

In [ ]:
import os

In [ ]:
if os.path.isdir("./Tweetsumm"):
  !rm -r Tweetsumm
!git clone https://github.com/s295103/Tweetsumm.git

Cloning into 'Tweetsumm'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 83 (delta 48), reused 5 (delta 2), pack-reused 0
Receiving objects: 100% (83/83), 471.41 KiB | 18.86 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
if os.path.isdir("./XSummarization"):
  !rm -r XSummarization
!git clone https://github.com/s295103/XSummarization.git

Cloning into 'XSummarization'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 13), reused 34 (delta 10), pack-reused 0
Receiving objects: 100% (37/37), 454.64 KiB | 19.77 MiB/s, done.
Resolving deltas: 100% (13/13), done.


#Download Tweetsumm TWCS file

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp Tweetsumm/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download thoughtvector/customer-support-on-twitter
!unzip customer-support-on-twitter.zip -d Tweetsumm/data


100% 168M/169M [00:08<00:00, 23.8MB/s]
100% 169M/169M [00:08<00:00, 20.3MB/s]
Archive:  customer-support-on-twitter.zip
  inflating: Tweetsumm/data/sample.csv  
  inflating: Tweetsumm/data/twcs/twcs.csv  


In [ ]:
#Install COMET requirements

In [ ]:
!bash XSummarization/Comet/download_model.sh
!pip install -U sentence-transformers

import torch
if torch.cuda.is_available():
  device = "cuda"
  !pip install -r XSummarization/Comet/requirement-gpu.txt
else:
  device = "cpu"
  !pip install -r XSummarization/Comet/requirement-cpu.txt


--2024-02-18 16:38:11--  https://storage.googleapis.com/ai2-mosaic-public/projects/mosaic-kgs/comet-atomic_2020_BART.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1507095346 (1.4G) [application/zip]
Saving to: ‘comet-atomic_2020_BART.zip’

comet-atomic_2020_B 100%[===================>]   1.40G  23.5MB/s    in 64s     

2024-02-18 16:39:15 (22.5 MB/s) - ‘comet-atomic_2020_BART.zip’ saved [1507095346/1507095346]

Archive:  comet-atomic_2020_BART.zip
   creating: comet-atomic_2020_BART/
  inflating: comet-atomic_2020_BART/added_tokens.json  
  inflating: comet-atomic_2020_BART/.DS_Store  
  inflating: __MACOSX/comet-atomic_2020_BART/._.DS_Store  
  inflating: comet-atomic_2020_BART/tokenizer_config.json  
  inflating: comet-atomic_2020_BART/special_tokens_map.json  
  

#Define preprocessing function

In [ ]:
from Tweetsumm.tweet_sum_processor import TweetSumProcessor
from XSummarization.Comet.comet import Comet, generate_commonsense
from google.colab import files
import json
import warnings
from tqdm.notebook import tqdm

def sick_preprocessing(twcs_file, input_data, output_data, commonsense_dialog, commonsense_summ=None):
  warnings.filterwarnings('ignore')
  processor = TweetSumProcessor(twcs_file)
  comet = Comet("./comet-atomic_2020_BART")
  comet.model.zero_grad()

  with open(input_data, "r") as f:
    dialog_with_summaries = processor.get_dialog_with_summaries(f.readlines())

  with open(output_data, "w") as f:
    print("Refactoring dialogue data...")
    dialog_json = [d.get_json() for d in dialog_with_summaries]
    f.writelines("\n".join(dialog_json))

  with open(commonsense_dialog, "w") as f:
    print("Generating commonsense data for dialogues...")
    for d in tqdm(dialog_with_summaries):
      dialog = d.get_dialog()
      turns = [str(t) for t in dialog.get_turns()]
      commonsense = [generate_commonsense(comet, t) for t in turns]
      line = {
          "dialog_id": dialog.get_dialog_id(),
          "commonsense":commonsense
        }
      f.write(json.dumps(line))

  if commonsense_summ:
    with open(commonsense_summ, "w") as f:
      print("Generating commonsense data for summaries...")
      for d in tqdm(dialog_with_summaries):
        dialog = d.get_dialog()
        summaries = [str(s) for s in d.get_abstractive_summaries()]
        commonsense = [generate_commonsense(comet, s) for s in summaries]
        line = {
           "dialog_id": dialog.get_dialog_id(),
           "commonsense":commonsense
         }
        f.write(json.dumps(line))


#Apply preprocessing to the data

In [ ]:
if not os.path.isdir("/content/clean_data"):
  !mkdir clean_data
if not os.path.isdir("/content/Comet_data"):
  !mkdir Comet_data
  !mkdir Comet_data/tweetsumm
  !mkdir Comet_data/tweetsumm/dialogue
  !mkdir Comet_data/tweetsumm/summary

TWCS_FILE_PATH = "/content/Tweetsumm/data/twcs/twcs.csv"
TRAIN_FILE_PATH = "/content/Tweetsumm/tweet_sum_data_files/final_train_tweetsum.jsonl"
VALID_FILE_PATH = "/content/Tweetsumm/tweet_sum_data_files/final_valid_tweetsum.jsonl"
TEST_FILE_PATH = "/content/Tweetsumm/tweet_sum_data_files/final_test_tweetsum.jsonl"

CLEAN_TRAIN_FILE_PATH = "/content/clean_data/tweetsumm_train.json"
CLEAN_VALID_FILE_PATH = "/content/clean_data/tweetsumm_validation.json"
CLEAN_TEST_FILE_PATH = "/content/clean_data/tweetsumm_test.json"

COMET_DIALOGUE_TRAIN_FILE_PATH = "/content/Comet_data/tweetsumm/dialogue/comet_train.json"
COMET_DIALOGUE_VALID_FILE_PATH = "/content/Comet_data/tweetsumm/dialogue/comet_validation.json"
COMET_DIALOGUE_TEST_FILE_PATH = "/content/Comet_data/tweetsumm/dialogue/comet_test.json"
COMET_SUMMARY_TRAIN_FILE_PATH = "/content/Comet_data/tweetsumm/summary/comet_train.json"

print("Processing training data")
sick_preprocessing(TWCS_FILE_PATH, TRAIN_FILE_PATH, CLEAN_TRAIN_FILE_PATH, COMET_DIALOGUE_TRAIN_FILE_PATH, COMET_SUMMARY_TRAIN_FILE_PATH)

print("Processing validation data")
sick_preprocessing(TWCS_FILE_PATH, VALID_FILE_PATH, CLEAN_VALID_FILE_PATH, COMET_DIALOGUE_VALID_FILE_PATH)

print("Processing test data")
sick_preprocessing(TWCS_FILE_PATH, TEST_FILE_PATH, CLEAN_TEST_FILE_PATH, COMET_DIALOGUE_TEST_FILE_PATH)


Processing training data


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Refactoring dialogue data...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Generating commonsense data for dialogues...


  0%|          | 0/110 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Download data

In [ ]:
!zip -r tweetsumm_clean.zip clean_data
files.download("tweetsumm_clean.zip")
!zip -r tweetsumm_comet.zip Comet_data
files.download("tweetsumm_comet.zip")

  adding: clean_data/ (stored 0%)
  adding: clean_data/tweetsumm_validation.json (deflated 79%)
  adding: clean_data/tweetsumm_test.json (deflated 78%)
  adding: clean_data/tweetsumm_train.json (deflated 78%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: Comet_data/ (stored 0%)
  adding: Comet_data/tweetsumm/ (stored 0%)
  adding: Comet_data/tweetsumm/dialogue/ (stored 0%)
  adding: Comet_data/tweetsumm/dialogue/comet_train.json (deflated 82%)
  adding: Comet_data/tweetsumm/dialogue/comet_test.json (deflated 82%)
  adding: Comet_data/tweetsumm/dialogue/comet_validation.json (deflated 82%)
  adding: Comet_data/tweetsumm/summary/ (stored 0%)
  adding: Comet_data/tweetsumm/summary/comet_train.json (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>